In [1]:
# My Standard Spark Session!

# Python libraries:
import os
import sys
import re
from dateutil import parser
# import datetime
from datetime import datetime
from datetime import date
import builtins
import json
import functools
import operator
from itertools import product

# Numpy & Pandas!
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 80
#('display.max_colwidth', 80)


# Spark!
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession, Row


spark = SparkSession.builder.appName("myapp").getOrCreate()

#     spark = SparkSession.builder.master("yarn")\
#     .config("spark.executor.instances", "32")\
#     .config("spark.executor.cores", "4")\
#     .config("spark.executor.memory", "4G")\
#     .config("spark.driver.memory", "4G")\
#     .config("spark.executor.memoryOverhead","4G")\
#     .config("spark.yarn.queue","Medium")\
#     .appName("myapp")\
#     .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.debug.maxToStringFields","true")

In [2]:
%load_ext autoreload
%autoreload 2
# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload


# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)


from shared.app_context import *
from builder.DataFrameBuild import *

ctx = ApplicationContext("Dev-Job")

DFB = DataFrameBuild(ctx.spark)

print("%16s  %s" % ("Python Version:",sys.version))
print("%16s  %s" % ("Python Path:",os.path.dirname(sys.executable)))
print("%16s  %s" % ("My Python Libs:",my_library))
print("%16s  %s" % ("My Spark Dir:",my_spark))
print("%16s  %s" % ("My Spark Ctx:",ctx.spark))
# print(ctx.spark)
# print(os.listdir(my_spark))
# print(sys.path)
# print("\n")

 Python Version:  3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
    Python Path:  C:\Users\d810216\AppData\Local\conda\conda\envs\my_root
 My Python Libs:  C:\Users\d810216/.myconfigs
   My Spark Dir:  C:\Users\d810216/spark2_dfanalysis
   My Spark Ctx:  <pyspark.sql.session.SparkSession object at 0x0000025EE05DD7B8>


### 2 Dataframe comparisons:

In [3]:
def column_comparison(df1,df2):
    '''
    Get the column names present in both dataframes.
    :param df1: first dataframe
    :param df2: second dataframe
    '''
    df1_cols = df1.columns
    df2_cols = df2.columns
    only1_cols = list(set(df1_cols) - set(df2_cols))
    only2_cols = list(set(df2_cols) - set(df1_cols))
    common = list(set(df1_cols).intersection(set(df2_cols)))
    print("df1:\t",only1_cols)
    print("df2:\t",only2_cols)
    print("\nboth:\t",common)
    return common

In [4]:
# Count Partition Elements:
def count_in_a_partition(idx, iterator):
    count = 0
    for _ in iterator:
        count += 1
    return idx, count

# Number of partitions!
def df_partition_count(df):
    print("Number of Partitions:",df.rdd.getNumPartitions())
    print("count: ",df.count())
    list = df.rdd.mapPartitionsWithIndex(count_in_a_partition).collect()
    # print(type(list))
    arr_x = np.array(list)
    x2 = arr_x.shape[0]/2
    print(arr_x.shape,x2)
    arr_x2 = np.reshape(arr_x,(int(x2),2))
    print(arr_x2.shape)
    print(arr_x2)

In [5]:
num = 25000

def get_distributions(num):
    indices = np.random.randint(100000,999999,num)
    normal_dist  = np.random.standard_normal(num)
    prices = np.random.random(num) * 20000 + 20000 # 20,000 -> 40,000
    return indices, normal_dist, prices

# match:
Ai, An, Ap = get_distributions(num)

# mismatches:
Bi, Bn, Bp = get_distributions(2500)
Ci, Cn, Cp = get_distributions(1750)

# print(i)
# print(n)
# print(p)

ABi = np.concatenate([i,i1])
ABn = np.concatenate([n,n1])
ABp = np.concatenate([p,p1])

ACi = np.concatenate([i,i2])
ACn = np.concatenate([n,n2])
ACp = np.concatenate([p,p2])

# Let's create 2 different dataframes
# df1: have 27500 records
# df2: have 26750 records
# 25000 will definitely overlap, some will be duplicate, some will not match ~up to 1750 or so.

df1 = DFB.arrays_to_dataframe([ABi,ABn,ABp],['index','normal','prices'])

df2 = DFB.arrays_to_dataframe([ACi,ACn,ACp],['index','normal','prices'])

NameError: name 'i' is not defined

In [82]:
df1.limit(4).toPandas()

,index,normal,prices
0,556745,-0.67,"28,797.60"
1,886448,-1.44,"39,042.30"
2,249250,0.47,"35,044.79"
3,388097,1.14,"31,028.21"


### join!

In [125]:
df_c0 = df1.alias("a")\
.join(df2.alias("b"),col("a.index") == col("b.index"),"inner")\
.drop(col("b.index"))

In [126]:
print(df_c0.count())

25823


In [127]:
df_c1 = df1.alias("a")\
.join(df2.alias("b"),col("a.index") != col("b.index"),"inner")\
.drop(col("b.index"))

In [128]:
print(df_c1.count())

735599177


In [130]:
df_c2 = df1.alias("a")\
.join(df2.alias("b"),col("a.index") == col("b.index"),"left_outer")\
.drop(col("b.index"))

In [131]:
print(df_c2.count())

28238


In [132]:
df_c3 = df1.alias("a")\
.join(df2.alias("b"),col("a.index") == col("b.index"),"right_outer")\
.drop(col("b.index"))

In [133]:
print(df_c3.count())

27516


In [134]:
df_c4 = df2.alias("a")\
.join(df1.alias("b"),col("a.index") == col("b.index"),"right_outer")\
.drop(col("b.index"))

In [135]:
print(df_c4.count())

28238


In [136]:
df_c5 = df1.alias("a")\
.join(df2.alias("b"),col("a.index") == col("b.index"),"full_outer")\
.drop(col("b.index"))

In [137]:
print(df_c5.count())

29931


### Let's find out exactly which ~800-850 records overlapped.

In [108]:
A = [1,1,2,3,5,7,8,8,9,11,11]
B = [1,2,7,8,9,10,11,11,11]
C = A + B

In [117]:
def unique(a,b):
    # only in 1 or other
    x = list(set(a) - set(b))
    y = list(set(b) - set(a))
    return list(x+y)

def union(a,b):
    # set(A+B)
    return list(set(a) | set(b))

def intersect(a,b):
    return list(set(a) & set(b))

def overlap(a,b):
    z = list(a + b)
    overlaps = []
    for x in z:
        if((x in a) & (x in b)):
            print(x)
            overlaps.append(x)
            a.remove(x)
            b.remove(x)
    return overlaps
            

        
#     c = [x for x in a if x in b]
#     d = [x for x in b if x in a]
#     print(c)
#     print(d)
#     return c

def combine(a,b):
    #c = a + b
    c = a - b
    d = b - a
    return list(c + d)
    #d = [x for x in c] if c in b]
    #d = [d for d in b if d in a]
    #return c + d

In [121]:
ab_unq = unique(A,B)
ab_union = union(A,B)
ab_inter = intersect(A,B)

In [122]:
ab_combine = overlap(A,B)

In [123]:
print(ab_unq)
print(ab_union)
print(list(set(C)))
print(ab_inter)
print(ab_combine)

[8, 1, 3, 5, 10, 11]
[1, 3, 5, 8, 10, 11]
[1, 2, 3, 5, 7, 8, 9, 10, 11]
[]
[]


In [89]:
# lst ai - bi
# lst bi - ai
lst_ai = list(set(ai) - set(bi))
lst_bi = list(set(bi) - set(ai))

print(len(i1) - len(lst_ai))
print(len(i2) - len(lst_bi))

# intersect:
print(len(list(set(ai) & set(bi))))



92
59
24666


In [42]:
# overlap / intersect:
lst_overlaps = list(set(ai) & set(bi))
print(len(lst_overlaps))
print(lst_overlaps)

24666
[131072, 655373, 655377, 917523, 524320, 393248, 131108, 917550, 786482, 131135, 393283, 393284, 262213, 393285, 917577, 917579, 131152, 655446, 917594, 524384, 262241, 393318, 524394, 917614, 393330, 655482, 393344, 786572, 917652, 393371, 131228, 393391, 393397, 524471, 917692, 786620, 393429, 655578, 262363, 786653, 524518, 917740, 262380, 917750, 393466, 262402, 131331, 524559, 655634, 786706, 262425, 786717, 393505, 786725, 917800, 917803, 262447, 131377, 917811, 524598, 655677, 655682, 393545, 262483, 393559, 131422, 786785, 786793, 393590, 524670, 262534, 131470, 393622, 524702, 655777, 786862, 917935, 262575, 655805, 655807, 524738, 786895, 655847, 524785, 131570, 786939, 131585, 918031, 786968, 131613, 918047, 393760, 131620, 262694, 131624, 655918, 655944, 787030, 918102, 524891, 918117, 787049, 918121, 262770, 262771, 655989, 918138, 262795, 524944, 787091, 918180, 393899, 393907, 131763, 656061, 131775, 393920, 918222, 656086, 918232, 262874, 393953, 262889, 131819, 7

In [46]:
lst_union25 = list(set(ai))
a = len(lst_union25)

lst_union = list(set(ai) & set(bi))
b = len(lst_union)
# print(lst_union)

print(a)
print(b)

lst_unionJ = list(set(lst_union25) | set(lst_union))
print(len(lst_unionJ))

27074
24666
27074


In [ ]:
df_combined = df1.alias("a")\
.join(df2.alias("b"),col("a.index") == col("b.index"),"inner")\
.drop(col("b.index"))

In [130]:
df_sales.limit(8).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep
4,1677,lwp,625,5,22,2011,8,1,39,"61,863.09",Volkswagon
5,1697,wvs,726,3,27,2019,12,8,32,"65,485.91",Tesla
6,1806,zlp,516,5,24,2018,17,40,4,"97,498.34",Hyundai
7,1950,gjk,570,1,24,2013,1,4,21,"50,618.46",Volkswagon


### Create a unique identifier column

In [131]:
unique_id = [col('make'),lit('_'),col('vin'), lit('_'), col('year')]

df_id = df_sales.withColumn('key',concat(*unique_id))

In [132]:
df_id.limit(8).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make,key
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep,Jeep_nab_2010
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford,Ford_tpo_2013
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla,Tesla_dow_2012
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep,Jeep_gxa_2014
4,1677,lwp,625,5,22,2011,8,1,39,"61,863.09",Volkswagon,Volkswagon_lwp_2011
5,1697,wvs,726,3,27,2019,12,8,32,"65,485.91",Tesla,Tesla_wvs_2019
6,1806,zlp,516,5,24,2018,17,40,4,"97,498.34",Hyundai,Hyundai_zlp_2018
7,1950,gjk,570,1,24,2013,1,4,21,"50,618.46",Volkswagon,Volkswagon_gjk_2013


### Create some Date columns

#### For this, I plan to use some UDF's (user defined functions)

In [205]:
# UDF's with lambdas!


# functions:
def leading_zeroes(x,width=2):
    return str(x).zfill(width)

def nohyphens(z):
    return re.sub("-","",str(z))

def date_from_str_mdY(z):
    return datetime.strptime(z,'%m-%d-%Y')

def datetime_from_str_mdY_time(z):
    return datetime.strptime(z,'%m-%d-%Y %H:%M:%S')
    # dt = datetime.strptime("21/11/06 16:30", "%d/%m/%y %H:%M")


# udf's:
udf_leading_zeroes2 = udf(lambda z: leading_zeroes(z,2),StringType())
udf_leading_zeroes4 = udf(lambda z: leading_zeroes(z,4),StringType())
udf_leading_zeroes6 = udf(lambda z: leading_zeroes(z,6),StringType())

# NB: the modified but dual use of "nohyphens"
udf_nohyphen_str = udf(lambda z: nohyphens(z),StringType())
udf_nohyphen_int = udf(lambda z: int(nohyphens(z)),IntegerType())

# date, datetime:
udf_date_from_str_mdY = udf(lambda z: date_from_str_mdY(z),DateType())
udf_datetime_from_str_mdY_time = udf(lambda z: datetime_from_str_mdY_time(z),TimestampType())

In [198]:
print(leading_zeroes('67',6))

000067


### Create a DateType with UDF

#### All at once

In [182]:
# just add date column

df_finaldate = df_id\
.withColumn("month-2",udf_leading_zeroes2(col("month")))\
.withColumn("day-2",udf_leading_zeroes2(col("day")))\
.withColumn("year-4",udf_leading_zeroes4(col("year")))\
.withColumn("date-hyphen",concat("month-2",lit('-'),"day-2",lit('-'),"year-4"))\
.withColumn("date",udf_date_from_str_mdY(col("date-hyphen")))\
.drop("month-2","day-2","year-4","date-hyphen")

In [183]:
# df_finaldate.printSchema()
df_finaldate.limit(4).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make,key,date
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep,Jeep_nab_2010,2010-10-15
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford,Ford_tpo_2013,2013-11-15
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla,Tesla_dow_2012,2012-05-08
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep,Jeep_gxa_2014,2014-10-12


#### Step by step

In [184]:
# let's convert month-day-year into 08-24-2012 into a new column as a string, integer, and date
df_dated1 = df_id\
.withColumn("month-2",udf_leading_zeroes2(col("month")))\
.withColumn("day-2",udf_leading_zeroes2(col("day")))\
.withColumn("year-4",udf_leading_zeroes4(col("year")))\
.withColumn("year-6",udf_leading_zeroes6(col("year")))

In [185]:
df_dated1.limit(4).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make,key,month-2,day-2,year-4,year-6
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep,Jeep_nab_2010,10,15,2010,002010
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford,Ford_tpo_2013,11,15,2013,002013
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla,Tesla_dow_2012,05,08,2012,002012
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep,Jeep_gxa_2014,10,12,2014,002014


In [186]:
df_dated2 = df_id\
.withColumn("month-2",udf_leading_zeroes2(col("month")))\
.withColumn("day-2",udf_leading_zeroes2(col("day")))\
.withColumn("year-4",udf_leading_zeroes4(col("year")))\
.withColumn("date-hyphen",concat("month-2",lit('-'),"day-2",lit('-'),"year-4"))\
.drop("month-2","day-2","year-4")

In [187]:
df_dated2.limit(4).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make,key,date-hyphen
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep,Jeep_nab_2010,10-15-2010
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford,Ford_tpo_2013,11-15-2013
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla,Tesla_dow_2012,05-08-2012
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep,Jeep_gxa_2014,10-12-2014


In [188]:
# let's remove the hyphens now, and get a string, or integer back
df_nohyphen = df_dated2\
.withColumn("int-date",udf_nohyphen_int(col("date-hyphen")))\
.withColumn("str-date",udf_nohyphen_str(col("date-hyphen")))\
.withColumn("date",udf_date_from_str_mdY(col("date-hyphen")))

In [189]:
df_nohyphen.printSchema()

root
 |-- index: long (nullable = true)
 |-- vin: string (nullable = true)
 |-- division: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)
 |-- year: long (nullable = true)
 |-- hour: long (nullable = true)
 |-- minute: long (nullable = true)
 |-- second: long (nullable = true)
 |-- price: double (nullable = true)
 |-- make: string (nullable = true)
 |-- key: string (nullable = true)
 |-- date-hyphen: string (nullable = true)
 |-- int-date: integer (nullable = true)
 |-- str-date: string (nullable = true)
 |-- date: date (nullable = true)



In [190]:
df_nohyphen.limit(4).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make,key,date-hyphen,int-date,str-date,date
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep,Jeep_nab_2010,10-15-2010,10152010,10152010,2010-10-15
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford,Ford_tpo_2013,11-15-2013,11152013,11152013,2013-11-15
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla,Tesla_dow_2012,05-08-2012,5082012,05082012,2012-05-08
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep,Jeep_gxa_2014,10-12-2014,10122014,10122014,2014-10-12


In [199]:
# let's remove the hyphens now, and get a string, or integer back
df_date = df_dated2\
.withColumn("int-date",udf_nohyphen_int(col("date-hyphen")))\
.withColumn("str-date",udf_nohyphen_str(col("date-hyphen")))\
.withColumn("date",udf_date_from_str_mdY(col("date-hyphen")))\
.drop("int-date","str-date","date-hyphen")

In [200]:
df_date.limit(4).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make,key,date
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep,Jeep_nab_2010,2010-10-15
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford,Ford_tpo_2013,2013-11-15
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla,Tesla_dow_2012,2012-05-08
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep,Jeep_gxa_2014,2014-10-12


In [201]:
df_date.printSchema()

root
 |-- index: long (nullable = true)
 |-- vin: string (nullable = true)
 |-- division: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)
 |-- year: long (nullable = true)
 |-- hour: long (nullable = true)
 |-- minute: long (nullable = true)
 |-- second: long (nullable = true)
 |-- price: double (nullable = true)
 |-- make: string (nullable = true)
 |-- key: string (nullable = true)
 |-- date: date (nullable = true)



### Create TimestampType with UDF

In [215]:
# just add timestamp column

df_finaldatetime = df_id\
.withColumn("month-2",udf_leading_zeroes2(col("month")))\
.withColumn("day-2",udf_leading_zeroes2(col("day")))\
.withColumn("year-4",udf_leading_zeroes4(col("year")))\
.withColumn("hour-2",udf_leading_zeroes2(col("hour")))\
.withColumn("minute-2",udf_leading_zeroes2(col("minute")))\
.withColumn("second-2",udf_leading_zeroes2(col("second")))\
.withColumn("datetime-hyphen",concat("month-2",lit('-'),"day-2",lit('-'),"year-4",
                                     lit(" "),"hour-2",lit(":"),"minute-2",lit(":"),"second-2"))\
.withColumn("datetime",udf_datetime_from_str_mdY_time(col("datetime-hyphen")))\
.drop("month-2","day-2","year-4","datetime-hyphen","hour-2","minute-2","second-2")

In [216]:
df_finaldatetime.printSchema()

root
 |-- index: long (nullable = true)
 |-- vin: string (nullable = true)
 |-- division: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)
 |-- year: long (nullable = true)
 |-- hour: long (nullable = true)
 |-- minute: long (nullable = true)
 |-- second: long (nullable = true)
 |-- price: double (nullable = true)
 |-- make: string (nullable = true)
 |-- key: string (nullable = true)
 |-- datetime: timestamp (nullable = true)



In [217]:
df_finaldatetime.limit(4).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price,make,key,datetime
0,26,nab,105,10,15,2010,13,51,20,"15,714.75",Jeep,Jeep_nab_2010,2010-10-15 13:51:20
1,29,tpo,862,11,15,2013,21,47,5,"77,844.98",Ford,Ford_tpo_2013,2013-11-15 21:47:05
2,474,dow,864,5,8,2012,19,40,42,"34,483.40",Tesla,Tesla_dow_2012,2012-05-08 19:40:42
3,964,gxa,913,10,12,2014,14,16,4,"3,115.38",Jeep,Jeep_gxa_2014,2014-10-12 14:16:04
